In [1]:
import numpy as np

In [25]:
class Value:
    
    def __init__(self, data, _prev=(), _op='' ):
        self.data = data
        self._prev=set(_prev)
        self._op = _op
        self.grad = 0.0
        
    def __repr__(self):
        return f'data={self.data}'
    
    def __str__(self):
        return self.value
    
    def __add__(self, other):
        return Value(data=self.data + other.data, _prev=(self,other), _op='+')
    
    def __mul__(self, other):
        return Value(data=self.data * other.data, _prev=(self,other), _op='*')
    
    def tanh(self):
        x = self.data
        return Value((np.exp(2*x)-1)/ (np.exp(2*x)+1), _prev=(self,), _op="tanh")
    

In [26]:
### Creating a single neuron with two inputs, a bias and Tanh as a activation function

In [27]:
x1 = Value(2.0)
x2 = Value(0.0)

w1 = Value(-3.0)
w2 = Value(1.0)

b = Value(6.7)

# implementing x1w1 + x2w2 + b 

x1w1 = x1 * w1
x2w2 = x2 * w2

summation = x1w1 + x2w2
n = summation + b
n.tanh()

data=0.6043677771171636